In [1]:
import sys
sys.path.insert(0, '../') 

In [2]:
import rdflib
from wlkernel import *
import random
from itertools import chain
from pathlib import Path
from pprint import pprint
from collections import Counter

In [3]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [4]:
# there is only one instance that belong to the class id5instance, then we remove it
#Counter([t[1] for t in all_instances]).most_common()  # command to verify it, use it after all_instances definition
rdf_graph.remove( (
    rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1959instance'),
    rdflib.term.URIRef('http://swrc.ontoware.org/ontology#affiliation'),
    rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id5instance')
))

In [5]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
all_instances = [ (t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) ) ]
assert len(all_instances) == len(set(all_instances))

In [6]:
n = 40
random.seed(42)
instances = random.sample(all_instances, n)
instances_id = [x[0] for x in instances]
instances_class = [x[1] for x in instances]
# print('Amount of the values of the classes:\n\t')
# pprint(Counter(instances_class).most_common())

In [7]:
employs_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#employs')
head_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#head')
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')
predicates_to_remove = [employs_uri, head_uri, member_uri, affiliation_uri]
for predicate in predicates_to_remove:
    rdf_graph.remove( (None, predicate, None) )

In [8]:
instances_id = [str(x) for x in instances_id]
instances_class = [str(x) for x in instances_class]

In [9]:
######## number of cores: 12 | max_depth: 2 | iterations: 1 | seed: 42
########
#### number of instances: 5
########
# SEQ: 29.8 s ± 382 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# PAR: 10.9 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# SPEEDUP: 2.7339
########
#### number of instances: 10
########
# SEQ: 1min 6s ± 501 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# PAR:   28.5s ± 161 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
# SPEEDUP: 2.3571
########
#### number of instances: 20
########
# SEQ: 4min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# PAR: 1min 51s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SEEPDUP: 2.4324
########
#### number of instances: 30
########
# SEQ: 14min 32s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# PAR: 5min 12s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SPEEDUP: 2.7949
########
#### number of instances: 40
########
# SEQ: ???
# PAR: 8min 7s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
# SPEEDUP: ???
########


In [10]:
# %timeit -n 1 -r 1 kernel_matrix = compute_kernel_matrix(rdf_graph, instances_id, 2, 1)

14min 32s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
# %timeit -n 1 -r 1 kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)
kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:   45.4s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:   53.8s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:  

In [2]:
kernel_matrix == kernel_matrix_par

NameError: name 'kernel_matrix' is not defined

In [12]:
X = kernel_matrix_par
y = instances_class

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
tree_clf = RandomForestClassifier(n_estimators=75)
tree_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=75,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [16]:
y_pred_train = tree_clf.predict(X_train)
y_pred = tree_clf.predict(X_test)

In [17]:
def accuracy(y_true, y_pred): return sum([x == y for x, y in zip(y_true, y_pred)]) / len(y_true)

In [18]:
print('Train accuracy: ', accuracy(y_train, y_pred_train))
print('Test accuracy: ', accuracy(y_test, y_pred))

Train accuracy:  1.0
Test accuracy:  0.45


In [19]:
y_pred

array(['http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id3instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id4instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id2instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id4instance',
       'http://www.aifb.uni-karlsruhe.de/Forschungsgruppen/viewForschungsgruppeOWL/id1instance',
       'http://www.aifb.uni-ka